In [27]:
def check_params(schema, tablename):
    if schema == "" or schema is None: schema = "public"
    if tablename == "" or tablename is None: raise Exception(f"Tablename not defined")
    else: tablename = tablename.upper()
    return schema, tablename

def get_truncate_table(schema, tablename):
    return f"truncate table {schema}.{tablename}"

def get_create_table(schema, tablename, tablespace=None, interfaces=[]):
    if tablename == "segment":
        create_table = f"create table if not exists {schema}.{tablename}( seg_id integer NOT NULL, oneway boolean, name character varying(254), highway text[], length double precision, ver_id_star integer NOT NULL, ver_id_final integer NOT NULL )"
    if tablename == "TIME":
        create_table = f"create table if not exists {schema}.{tablename}( time_id integer NOT NULL, period character varying(32), weekday character varying(16), day integer, month integer, year integer)"
    if tablename == "vertice":
        create_table = f"create table if not exists {schema}.{tablename}( ver_id integer NOT NULL, vertice text NOT NULL, zone_id integer NOT NULL, district_id integer NOT NULL, neighborhood_id integer NOT NULL)"
    else:
        create_table = f"create table if not exists {schema}.{tablename}( time_id integer NOT NULL, seg_id integer NOT NULL )"
        for interface in interfaces:
            create_table = create_table.replace(")", f", {interface} integer)")
    
    if tablespace: create_table = create_table + f" tablespace {tablespace}"

    return create_table

def get_copy_table_by_csv(csv_file_path, schema, tablename):
    sql = f'''
        COPY {schema}.{tablename} FROM '{csv_file_path}' DELIMITER ';' CSV;
    '''
    return sql

def generate_csv_file_path(path, tablename, dataframe):
    csv_file_path = f"{path}/{tablename}.csv"
    dataframe.to_csv(csv_file_path, index=False, header=False, sep =';')

def execute_sql(urlconnect, sql):
    print(f"Open Connection!")
    engine = create_engine(urlconnect)
    conn = engine.connect()
    pg_conn = psycopg2.connect(urlconnect)
    cur = pg_conn.cursor()
    print(f"Start SQL execution: [{sql}]")
    cur.execute(sql)
    print(f"Finish SQL execution: [{sql}]")
    pg_conn.commit()
    cur.close()
    conn.close()
    print(f"Close Connection!")
    

In [29]:
import pandas as pd
import numpy as np
import psycopg2
from shapely import wkb
from sqlalchemy import create_engine

def sync_tablename(parquet_workdir, urlconnect, schema, tablename, tablespace=None, interfaces=[]):
    # Read database file
    parquet_file_path = f"{parquet_workdir}/{tablename}.parquet"
    dataframe = pd.read_parquet(parquet_file_path)

    generate_csv_file_path(parquet_workdir, tablename, dataframe)

    create_table = get_create_table(schema, tablename, tablespace, interfaces)
    execute_sql(urlconnect, create_table)

    truncate_table = get_truncate_table(schema, tablename)
    execute_sql(urlconnect, truncate_table)

    if tablename == "VERTICE":
        alter_table = f"alter table {schema}.{tablename} drop column if exists vertice"
        execute_sql(urlconnect, alter_table)
        alter_table = f"alter table {schema}.{tablename} add column vertice text not null"
        execute_sql(urlconnect, alter_table)
    elif tablename == "SEGMENT":
        dataframe['name']    = dataframe['name'].apply(lambda name : "{" + ",".join(name) + "}")
        dataframe['highway'] = dataframe['highway'].apply(lambda highway : "{" + ",".join(highway) + "}")
    else:
        for c in dataframe.columns: dataframe[c] = dataframe[c].astype(int)
    
    sql_copy = get_copy_table_by_csv(parquet_workdir, schema, tablename)
    execute_sql(urlconnect, sql_copy)

    if tablename == "VERTICE":
        execute_sql(urlconnect, "ALTER TABLE vertice add column geom geometry(point, 4326)")
        execute_sql(urlconnect, "UPDATE vertice SET geom = st_setsrid(st_makepoint(split_part(vertice, ',',1) ::double precision, split_part(vertice, ',',2)::double precision),4326)")
        execute_sql(urlconnect, "ALTER TABLE vertice drop column vertice")
        execute_sql(urlconnect, "ALTER TABLE vertice rename column geom to vertice")


'create table if not existis maiconbanni.CRIME( time_id integer NOT NULL, seg_id integer NOT NULL , a integer, b integer, c integer) tablespace default'

In [21]:
import pandas as pd
import psycopg2
from shapely import wkb
from sqlalchemy import create_engine
url = 'postgresql://cdadmin@localhost:5432/cdbase'
engine   = create_engine(url)
sql = "select * from public.crime"
conn = engine.connect()
conn.execute(sql)
conn.close()
pd.read_sql_query(sql, engine)

,time_id,seg_id,vehicles_rob,phones_rob,vehicles_steal,phones_steal,femicides,injurys_followed_by_death,intentional_homicides,robberys
0,1,3391,1,0,0,0,0,0,0,0
1,1,4501,0,1,0,0,0,0,0,0
2,1,5193,0,0,1,0,0,0,0,0
3,1,5247,0,1,0,0,0,0,0,0
4,1,6692,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
165707,3495,103416,0,0,0,0,0,0,1,0
165708,3496,287412,0,0,0,0,0,0,1,0
165709,3497,249767,0,0,0,0,0,0,1,0
165710,3498,103416,0,0,0,0,0,0,0,1
